In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
# dataset.tail()
# Check the columns in the dataset
print("Columns in the dataset:", dataset.columns.tolist())

# Strip any leading or trailing whitespace from column names
dataset.columns = dataset.columns.str.strip()




In [ ]:
# # Convert categorical data to numbers
# # One-hot encoding for 'sex', 'smoker', 'region'
# dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

# # Split dataset into features and labels
# labels = dataset.pop('expenses')
# # Split the dataset into training (80%) and testing (20%)
# train_dataset, test_dataset = np.split(dataset.sample(frac=1, random_state=42), [int(0.8 * len(dataset))])
# train_labels, test_labels = np.split(labels.sample(frac=1, random_state=42), [int(0.8 * len(labels))])

# # Normalize the features
# train_dataset = (train_dataset - train_dataset.mean()) / train_dataset.std()
# test_dataset = (test_dataset - test_dataset.mean()) / test_dataset.std()

# # Create the model
# model = keras.Sequential([
#     layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(1)  # Output layer
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# # Train the model
# history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)

# Convert categorical data to numbers
# Check if the columns exist
categorical_columns = ['sex', 'smoker', 'region']
missing_columns = [col for col in categorical_columns if col not in dataset.columns]
if missing_columns:
    raise KeyError(f"Missing columns in the dataset: {missing_columns}")

dataset = pd.get_dummies(dataset, columns=categorical_columns, drop_first=True)

# Split dataset into features and labels
labels = dataset.pop('expenses')
# Shuffle the dataset before splitting
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)
labels = labels.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into training (80%) and testing (20%)
split_index = int(0.8 * len(dataset))
train_dataset = dataset[:split_index]
test_dataset = dataset[split_index:]
train_labels = labels[:split_index]
test_labels = labels[split_index:]

# Normalize the features
train_dataset_mean = train_dataset.mean()
train_dataset_std = train_dataset.std()
train_dataset = (train_dataset - train_dataset_mean) / train_dataset_std
test_dataset = (test_dataset - train_dataset_mean) / train_dataset_std

# Create the model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# Train the model
history = model.fit(train_dataset, train_labels, epochs=200, validation_split=0.2, verbose=0)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
